# --- Day 16: Ticket Translation ---

https://adventofcode.com/2020/day/16

In [1]:
path = '../inputs/'

## Part 1

In [2]:
with open(path + 'example_train_ticket_notes.txt') as file:
    example_fields, example_my_ticket, example_nearby_tickets = \
        file.read().strip().split('\n\n')

In [3]:
with open(path + 'train_ticket_notes.txt') as file:
    fields, my_ticket, nearby_tickets = file.read().strip().split('\n\n')

In [4]:
import re

def get_valid_numbers(fields):

    valid_nums = set()
    field_nums = [f.split('-') for f in re.findall('\d+-\d+', fields)]
    
    for n in field_nums:
        for i in range(int(n[0]), int(n[1]) + 1):
            valid_nums.add(i)
    
    return valid_nums

In [5]:
def calc_ticket_error_rate(fields, nearby_tickets):
    error_rate = 0
    
    valid_nums = get_valid_numbers(fields)
    ticket_nums = re.findall('\d+', nearby_tickets)
    
    for n in ticket_nums:
        if not int(n) in valid_nums:
            error_rate += int(n)
    
    return error_rate

In [6]:
calc_ticket_error_rate(example_fields, example_nearby_tickets) # Should return 71

71

In [7]:
calc_ticket_error_rate(fields, nearby_tickets)

21956

## Part 2

In [8]:
with open(path + 'example2_train_ticket_notes.txt') as file:
    example2_fields, example2_my_ticket, example2_nearby_tickets = \
        file.read().strip().split('\n\n')

In [9]:
def remove_invalid_tickets(fields, nearby_tickets):
    valid_nums = get_valid_numbers(fields)
    ticket_rows = [row for row in nearby_tickets.split('\n') if not row.startswith('nearby')]

    valid_tickets = []
    
    for row in ticket_rows:
        delete = False
        for num in row.split(','):
            if not int(num) in valid_nums:
                delete = True
        if not delete:
            valid_tickets.append([int(n) for n in row.split(',')])
    
    return valid_tickets

In [10]:
def parse_fields(fields):
    """Return a dictionary with keys are field names and values are sets of valid values."""
    fields_dict = {}
    field_rows = fields.strip().split('\n')

    for row in field_rows:
        field_name, nums = row.split(':')
        fields_dict[field_name] = get_valid_numbers(nums)
    
    return fields_dict

In [11]:
def transpose_valid_tickets(valid_tickets):
    """Return valid nearby tickets as a list of columns sets."""

    # Transpose each column into a row containing a set
    col_sets = [set(i) for i in zip(*valid_tickets)]

    return col_sets

In [12]:
from collections import defaultdict
from pprint import pprint

def part_2(fields, my_ticket, nearby_tickets, field_startswith=''):
    
    # Set up the data
    fields_dict = parse_fields(fields)
    my_tic = [int(n) for n in my_ticket.split('\n')[1].split(',')]
    valid_tickets = remove_invalid_tickets(fields, nearby_tickets)
    valid_ticket_col_sets = transpose_valid_tickets(valid_tickets)
    
    # Assemple a dictionary containing possible matches for each key
    possible_matches = defaultdict(set)
    for key in fields_dict.keys():
        for i, col in enumerate(valid_ticket_col_sets):
            if col.issubset(fields_dict[key]):
                possible_matches[key].add(i)

    # Create a map dictionary for the appropriate field:column matches
    map = {}
    selected = set()
    
    # Loop across the columns in valid nearby tickets
    for i in range(1, len(valid_ticket_col_sets) + 1):
        for key in possible_matches.keys():
            # When len(possible_matches[key]) == 1, that's the only possible match
            if len(possible_matches[key]) == i:
                match = [x for x in possible_matches[key] if x not in selected][0]
                selected.add(match)
                map[key] = match

    pprint(map)       
                
    total = 1
    for key in map.keys():
        if key.startswith(field_startswith):
            total *= my_tic[map[key]]
    
    return total

In [13]:
part_2(example2_fields, 
       example2_my_ticket, 
       example2_nearby_tickets, 
       field_startswith='') # Should return 1716

{'class': 1, 'row': 0, 'seat': 2}


1716

In [14]:
part_2(fields, my_ticket, nearby_tickets, field_startswith='departure')

{'arrival location': 8,
 'arrival platform': 0,
 'arrival station': 16,
 'arrival track': 19,
 'class': 13,
 'departure date': 17,
 'departure location': 10,
 'departure platform': 4,
 'departure station': 18,
 'departure time': 15,
 'departure track': 14,
 'duration': 12,
 'price': 2,
 'route': 11,
 'row': 9,
 'seat': 5,
 'train': 6,
 'type': 1,
 'wagon': 3,
 'zone': 7}


3709435214239